In [5]:
import os
import pandas as pd
import torch
from sklearn.metrics import classification_report

In [6]:
root_dir = os.path.join(os.getcwd(), os.path.join( "../")) # MIGHT NEED TO CHANGE LATER
input_dir = os.path.join(root_dir, "input")
roam_path = os.path.join(input_dir, "Demo-Annotations-R1-2020-08-21.xlsx")
active_sheet = "Docs"
df = pd.read_excel(roam_path, sheet_name=active_sheet)

## df = pd.read_csv("Demo-Annotations-R1-2020-08-21.csv")

In [7]:
def process_text(s):
    s = s.replace("Claim1:\n\n", "")
    return s.split("\n\nClaim2:\n\n")

In [8]:
df['example'] = df['text'].apply(process_text)

In [9]:
def normalize_tags(s):
    s = s.replace("STRICT_", "")
    return s.lower()

In [10]:
df['label'] = df['tags'].apply(normalize_tags)

In [23]:
df.example.values[0]

['our findings reveal that nelfinavir is highly effective in inhibiting sars-cov-2 replication in vitro and has the high c max -ec 50 and c tough -ec 50 ratio.',
 'nelfinavir was predicted to be a potential inhibitor of 2019-ncov main protease by an integrative approach combining homology modelling, molecular docking and binding free energy calculation']

In [22]:
import os, ssl
if (not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None)):
    ssl._create_default_https_context = ssl._create_unverified_context

model = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')

Using cache found in /Users/dnsosa/.cache/torch/hub/pytorch_fairseq_master


Unable to build Cython components. Please make sure Cython is installed if the torch.hub model you are loading depends on it.


In [13]:
_ = model.eval()
X = df['example'].apply(lambda x: model.encode(*x))
pred_indices = [model.predict('mnli', ex).argmax() for ex in X]
to_str = {0: 'contradiction', 1: 'neutral', 2: 'entailment'}
preds = [to_str[c.item()] for c in pred_indices]
gold = df['label'].values
print(classification_report(gold, preds))

               precision    recall  f1-score   support

contradiction       0.75      0.15      0.25        20
   entailment       0.50      0.04      0.08        23
      neutral       0.34      0.95      0.51        21

     accuracy                           0.38        64
    macro avg       0.53      0.38      0.28        64
 weighted avg       0.53      0.38      0.27        64

